## Work
1. 請比較 SGD optimizer 不同的 momentum 及使用 nesterov 與否的表現

In [1]:
import os
import keras

# 本作業可以不需使用 GPU, 將 GPU 設定為 "無" (若想使用可自行開啟)
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# 資料前處理 - X 標準化
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# 資料前處理 -Y 轉成 onehot
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
"""Code Here
設定超參數
"""
LEARNING_RATE = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
EPOCHS = 50
BATCH_SIZE = 256
MOMENTUM = 0.95

In [9]:
results = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""
for lr in LEARNING_RATE:
    keras.backend.clear_session() # 把舊的 Graph 清掉
    print("Experiment with LR = %.6f" % (lr))
    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    optimizer = keras.optimizers.SGD(lr=lr, nesterov=True, momentum=MOMENTUM)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True)
    
    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["accuracy"]
    valid_acc = model.history.history["val_accuracy"]
    
    exp_name_tag = "exp-lr-%s" % str(lr)
    results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}



Experiment with LR = 0.100000
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [===========

Epoch 49/50
50000/50000 [==============================] - 6s 116us/step - loss: 1.5504 - accuracy: 0.4521 - val_loss: 1.6577 - val_accuracy: 0.4201
Epoch 50/50
50000/50000 [==============================] - 6s 115us/step - loss: 1.5492 - accuracy: 0.4520 - val_loss: 1.7276 - val_accuracy: 0.3996
Experiment with LR = 0.010000
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_____________________________________________________________

Epoch 47/50
50000/50000 [==============================] - 6s 120us/step - loss: 0.4141 - accuracy: 0.8507 - val_loss: 2.1005 - val_accuracy: 0.5373
Epoch 48/50
50000/50000 [==============================] - 6s 123us/step - loss: 0.3868 - accuracy: 0.8611 - val_loss: 2.2265 - val_accuracy: 0.5313
Epoch 49/50
50000/50000 [==============================] - 6s 118us/step - loss: 0.3983 - accuracy: 0.8570 - val_loss: 2.2146 - val_accuracy: 0.5340
Epoch 50/50
50000/50000 [==============================] - 6s 117us/step - loss: 0.3840 - accuracy: 0.8631 - val_loss: 2.2429 - val_accuracy: 0.5250
Experiment with LR = 0.001000
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
___________________________

Epoch 45/50
50000/50000 [==============================] - 6s 121us/step - loss: 0.9128 - accuracy: 0.6793 - val_loss: 1.3623 - val_accuracy: 0.5322
Epoch 46/50
50000/50000 [==============================] - 6s 121us/step - loss: 0.9063 - accuracy: 0.6824 - val_loss: 1.4051 - val_accuracy: 0.5282
Epoch 47/50
50000/50000 [==============================] - 6s 119us/step - loss: 0.8915 - accuracy: 0.6875 - val_loss: 1.4097 - val_accuracy: 0.5266
Epoch 48/50
50000/50000 [==============================] - 6s 119us/step - loss: 0.8828 - accuracy: 0.6895 - val_loss: 1.4255 - val_accuracy: 0.5234
Epoch 49/50
50000/50000 [==============================] - 6s 119us/step - loss: 0.8714 - accuracy: 0.6929 - val_loss: 1.4016 - val_accuracy: 0.5354
Epoch 50/50
50000/50000 [==============================] - 6s 123us/step - loss: 0.8588 - accuracy: 0.6980 - val_loss: 1.4620 - val_accuracy: 0.5156
Experiment with LR = 0.000100
Model: "model_1"
___________________________________________________________

Epoch 43/50
50000/50000 [==============================] - 6s 121us/step - loss: 1.5798 - accuracy: 0.4513 - val_loss: 1.5898 - val_accuracy: 0.4403
Epoch 44/50
50000/50000 [==============================] - 6s 122us/step - loss: 1.5754 - accuracy: 0.4521 - val_loss: 1.5898 - val_accuracy: 0.4400
Epoch 45/50
50000/50000 [==============================] - 6s 119us/step - loss: 1.5705 - accuracy: 0.4544 - val_loss: 1.5842 - val_accuracy: 0.4448
Epoch 46/50
50000/50000 [==============================] - 6s 120us/step - loss: 1.5660 - accuracy: 0.4549 - val_loss: 1.5800 - val_accuracy: 0.4421
Epoch 47/50
50000/50000 [==============================] - 6s 120us/step - loss: 1.5616 - accuracy: 0.4569 - val_loss: 1.5775 - val_accuracy: 0.4458
Epoch 48/50
50000/50000 [==============================] - 6s 117us/step - loss: 1.5583 - accuracy: 0.4581 - val_loss: 1.5719 - val_accuracy: 0.4475
Epoch 49/50
50000/50000 [==============================] - 6s 119us/step - loss: 1.5538 - accuracy: 0.4590

Epoch 41/50
50000/50000 [==============================] - 6s 125us/step - loss: 1.9748 - accuracy: 0.3129 - val_loss: 1.9711 - val_accuracy: 0.3143
Epoch 42/50
50000/50000 [==============================] - 6s 118us/step - loss: 1.9708 - accuracy: 0.3137 - val_loss: 1.9672 - val_accuracy: 0.3145
Epoch 43/50
50000/50000 [==============================] - 7s 132us/step - loss: 1.9669 - accuracy: 0.3160 - val_loss: 1.9633 - val_accuracy: 0.3171
Epoch 44/50
50000/50000 [==============================] - 7s 133us/step - loss: 1.9631 - accuracy: 0.3166 - val_loss: 1.9596 - val_accuracy: 0.3185
Epoch 45/50
50000/50000 [==============================] - 7s 134us/step - loss: 1.9594 - accuracy: 0.3180 - val_loss: 1.9560 - val_accuracy: 0.3193
Epoch 46/50
50000/50000 [==============================] - 7s 135us/step - loss: 1.9557 - accuracy: 0.3191 - val_loss: 1.9525 - val_accuracy: 0.3207
Epoch 47/50
50000/50000 [==============================] - 7s 136us/step - loss: 1.9522 - accuracy: 0.3206

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
color_bar = ["r", "g", "b", "y", "m", "k"]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()